## Google Gemini를 이용해 챗봇 만들고 네이버뉴스 검색후 요약하기

In [6]:
!pip install -q -U google-generativeai

In [7]:
!pip install python-dotenv

In [10]:
!pip install --upgrade pip

* .chatenv.txt 파일 : 나의 구글 gemini api key 들어있는 파일
* .chatenv 뒤에 .txt 지우기
* .chatenv 파일이 같은 08_AI_Serving 위치에 저장했는지 확인

In [19]:
import google.generativeai as genai
from dotenv import load_dotenv
import os
load_dotenv(".chatenv")

True

In [20]:
GOOGLE_API_KEY=os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [21]:
model = genai.GenerativeModel('gemini-1.5-flash')

In [22]:
response = model.generate_content("Write a story about a magic backpack.")
print(response.text)

The rain hammered against the attic window, echoing the drumbeat of Lila’s frustration. She was supposed to be packing for camp, but everything felt wrong. Her usual bright-colored clothes seemed dull, her favorite book too heavy, and her trusty flashlight just...ordinary. 

Suddenly, a gust of wind rattled the window, revealing a dusty corner she'd never noticed before. Behind a cobweb-draped trunk, a faded, leather backpack lay forgotten. Curiosity piqued, Lila brushed away the dust. The leather felt strangely warm, and an intricate silver clasp gleamed in the dim light.

Hesitantly, she opened the backpack. Inside, nestled on a bed of velvet, was a single, shimmering feather. It pulsated with an ethereal light, and as Lila touched it, a whisper of wind swirled around her, carrying a faint, sweet scent.

With a jolt, she realized she was no longer in the attic. She stood on a grassy hill, overlooking a shimmering turquoise lake. Birds chirped in a symphony of colors, and the air humm

In [32]:
# 최종 코드 

import os
import requests
import gradio as gr
from dotenv import load_dotenv

# .chatenv 파일에서 API 키 불러오기
load_dotenv(".chatenv")  # .chatenv 파일을 불러옵니다.
NAVER_CLIENT_ID = os.getenv('NAVER_CLIENT_ID')  # 네이버 클라이언트 ID
NAVER_CLIENT_SECRET = os.getenv('NAVER_CLIENT_SECRET')  # 네이버 클라이언트 시크릿 키

# 네이버 뉴스 검색 API 함수
def search_naver_news(query):
    api_url = "https://openapi.naver.com/v1/search/news.json"
    headers = {
        "X-Naver-Client-Id": NAVER_CLIENT_ID,
        "X-Naver-Client-Secret": NAVER_CLIENT_SECRET,
    }
    params = {
        "query": query,
        "display": 5,  # 검색 결과 개수
        "sort": "date"  # 최신 뉴스 정렬
    }
    
    response = requests.get(api_url, headers=headers, params=params)
    
    if response.status_code == 200:
        news_items = response.json().get('items', [])
        if not news_items:
            return "No news found."

        news_summary = []
        for item in news_items:
            title = item['title'].replace("<b>", "").replace("</b>", "")  # HTML 태그 제거
            link = item['link']
            news_summary.append(f"{title} - {link}")
        
        return "\n\n".join(news_summary)
    else:
        return f"Error {response.status_code}: {response.text}"

# Gradio 인터페이스 구성
with gr.Blocks() as app:
    gr.Markdown("# AI Chatbot with Naver News Summary")

    # 질문과 답변 챗봇
    with gr.Row():
        with gr.Column():
            question = gr.Textbox(label="Ask a question")
            answer = gr.Textbox(label="Answer")
            ask_btn = gr.Button("Ask")
            ask_btn.click(fn=generate_answer, inputs=question, outputs=answer)

        with gr.Column():
            news_query = gr.Textbox(label="Search Naver News")
            news_output = gr.Textbox(label="Naver News Results")
            news_btn = gr.Button("Search News")
            news_btn.click(fn=search_naver_news, inputs=news_query, outputs=news_output)

# Gradio 앱 실행
app.launch(inline=False, share=True)


Running on local URL:  http://127.0.0.1:7868
IMPORTANT: You are using gradio version 3.40.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://f05aa6329c267b17aa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
